In [12]:
from sklearn.linear_model import LogisticRegression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from optbinning import OptimalBinning
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import optuna
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, make_scorer
import random

(CVXPY) Mar 19 09:41:21 AM: Encountered unexpected exception importing solver CLARABEL:
ImportError('DLL load failed while importing clarabel: The specified procedure could not be found.')


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import importlib

In [2]:
#Let's load the data
# Carguemos los datos:
data = pd.read_csv('loanpred_train.csv')
data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001535,Male,No,0,Graduate,No,3254,0.0,50.0,360.0,1.0,Urban,Y
1,LP001792,Male,Yes,1,Graduate,No,3315,0.0,96.0,360.0,1.0,Semiurban,Y
2,LP002443,Male,Yes,2,Graduate,No,3340,1710.0,150.0,360.0,0.0,Rural,N
3,LP002517,Male,Yes,1,Not Graduate,No,2653,1500.0,113.0,180.0,0.0,Rural,N
4,LP001894,Male,Yes,0,Graduate,No,2620,2223.0,150.0,360.0,1.0,Semiurban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,LP001744,Male,No,0,Graduate,No,2971,2791.0,144.0,360.0,1.0,Semiurban,Y
487,LP002138,Male,Yes,0,Graduate,No,2625,6250.0,187.0,360.0,1.0,Rural,Y
488,LP001068,Male,Yes,0,Graduate,No,2799,2253.0,122.0,360.0,1.0,Semiurban,Y
489,LP001327,Female,Yes,0,Graduate,No,2484,2302.0,137.0,360.0,1.0,Semiurban,Y


In [3]:
categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
numerical_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']

target = 'Loan_Status'

In [4]:
data['Loan_Status'] = data['Loan_Status'].map({'Y': 1, 'N': 0})

In [5]:
# Imputación y codificación
data_pipeline = ColumnTransformer([
    ('num_imputer', SimpleImputer(strategy='median'), numerical_features),
    ('cat_imputer', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder())
    ]), categorical_features)
])

# Aplicar pipeline a los datos
X = data_pipeline.fit_transform(data.drop(columns=[target]))
y = data[target]

# División en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# Define Fitness Function Example
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def fitness_function(features):
    # Train Random Forest classifier using selected features
    model = RandomForestClassifier()
    model.fit(X_train[:, features], y_train)
    
    # Evaluate model performance on validation set
    y_pred = model.predict(X_test[:, features])
    accuracy = f1_score(y_test, y_pred)
    
    return accuracy

In [ ]:
# Define Genetic Operators Example
def selection(population, fitness_values):
    # Roulette wheel selection
    probabilities = fitness_values / np.sum(fitness_values)
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    return population[selected_indices]

def crossover(parents):
    # Single-point crossover
    crossover_point = np.random.randint(1, len(parents[0]))
    offspring = np.empty_like(parents)
    for i in range(len(parents)):
        parent1, parent2 = parents[np.random.choice(len(parents), size=2, replace=False)]
        offspring[i] = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return offspring

def mutation(offspring, mutation_rate):
    # Bit-wise mutation
    for i in range(len(offspring)):
        for j in range(len(offspring[i])):
            if np.random.rand() < mutation_rate:
                offspring[i][j] = 1 - offspring[i][j]
    return offspring

In [ ]:
# Genetic Algorithm Framework Example
class GeneticAlgorithm:
    def __init__(self, population_size, chromosome_length, fitness_function):
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.fitness_function = fitness_function
        self.population = self.initialize_population()
    
    def initialize_population(self):
        # Initialize population randomly
        return np.random.randint(2, size=(self.population_size, self.chromosome_length))
    def evolve(self, num_generations, mutation_rate):
        for _ in range(num_generations):
            # Evaluate fitness of current population
            fitness_values = [self.fitness_function(chromosome) for chromosome in self.population]
            
            # Select parents
            selected_population = selection(self.population, fitness_values)
            
            # Apply crossover
            offspring = crossover(selected_population)
            
            # Apply mutation
            mutated_offspring = mutation(offspring, mutation_rate)
            
            # Update population
            self.population = mutated_offspring
# Usage Example
population_size = 100
chromosome_length = 11
num_generations = 30
mutation_rate = 0.01
genetic_algorithm = GeneticAlgorithm(population_size, chromosome_length, fitness_function)
genetic_algorithm.evolve(num_generations, mutation_rate)

In [11]:
# Obtener índices de características seleccionadas
selected_features = np.where(genetic_algorithm.population[0] == 1)[0]
print(f"Índices de las variables seleccionadas: {selected_features}")

Índices de las variables seleccionadas: [1 2 4 5 7 8 9]


In [15]:
# Filtrar los datos con las características seleccionadas
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

# Implementar Knowledge Base Agent con estructura Tell-Ask
class KnowledgeBase:
    def __init__(self):
        self.knowledge = {}

    def tell(self, model_name, score):
        """Almacena el modelo y su F1-score en la Knowledge Base."""
        self.knowledge[model_name] = score

    def ask(self, query):
        """Devuelve el mejor modelo según el criterio de F1-score."""
        if query == "best_model":
            return max(self.knowledge, key=self.knowledge.get) if self.knowledge else None
        return self.knowledge.get(query, None)

# Definir modelos a evaluar
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
    "LogisticRegression": LogisticRegression()
}

# Inicializar Knowledge Base
kb = KnowledgeBase()

# Evaluar modelos con las características seleccionadas
for name, model in models.items():
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    score = f1_score(y_test, y_pred)
    kb.tell(name, score)

# Consultar el mejor modelo
best_model = kb.ask("best_model")
print(f"El mejor modelo seleccionado es: {best_model} con F1-score de {kb.ask(best_model)}")

El mejor modelo seleccionado es: LogisticRegression con F1-score de 0.8627450980392157


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:158: UserWarning: [10:50:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
